In [9]:
import numpy as np
import pandas as pd
from PIL import Image
# import potrace
from collections import deque
from tqdm import tqdm
import json


In [2]:
gate_list = ["camping0","camping1","camping2","camping3","camping4","camping5","camping6","camping7","camping8","entrance0","entrance1","entrance2","entrance3","entrance4","gate0","gate1","gate2","gate3","gate4","gate5","gate6","gate7","gate8","general-gate0","general-gate1","general-gate2","general-gate3","general-gate4","general-gate5","general-gate6","general-gate7","ranger-base","ranger-stop0","ranger-stop1","ranger-stop2","ranger-stop3","ranger-stop4","ranger-stop5","ranger-stop6","ranger-stop7"]

In [3]:
gate_idx = {}
for i, g in enumerate(gate_list):
    gate_idx[g] = i

In [4]:
p = Image.open('Lekagul Roadways.bmp')
p_array = np.array(p)
p_norm = p_array / np.max(p_array)

gates = pd.read_csv("gate_loc.csv")
# gates['t'] = [0] * 9 + [1] * 5 + [2] * 9 + [3] * 8 + [4] * 1 + [5] * 8
gates.r -= 1
gates.c -= 1
gates_dict = {}
gates_dict2 = {}
for i in range(gates.shape[0]):
    gates_dict[gates.gates[i]] = (gates.r[i], gates.c[i])
    gates_dict2[gates.gates[i]] = (gates.r[i], gates.c[i], gates.t[i])

hist = pd.read_csv("processed_data.csv")
all_edges = set()
for i in range(hist.shape[0]):
    if not pd.isnull(hist.next_gate[i]):
        all_edges.add(str(gate_idx[hist['gate-name'][i]]) + '-' + str(gate_idx[hist['next_gate'][i]]))
unique_edges = list(all_edges)
edge_set = set()
edge_dict = dict()
for edge in unique_edges:
    node1, node2 = edge.split("-")
    edge1 = node1 + '-' + node2
    edge2 = node2 + '-' + node1
    if edge1 not in edge_set and edge2 not in edge_set:
        edge_set.add(edge1)
        edge_dict[edge1] = edge1
        edge_dict[edge2] = edge1

In [15]:
edge_dict['30-4']

'30-4'

In [12]:
json.dumps(edge_dict)

'{"20-14": "20-14", "14-20": "20-14", "0-2": "0-2", "2-0": "0-2", "0-30": "0-30", "30-0": "0-30", "28-18": "28-18", "18-28": "28-18", "1-8": "1-8", "8-1": "1-8", "23-25": "23-25", "25-23": "23-25", "1-28": "1-28", "28-1": "1-28", "26-18": "26-18", "18-26": "26-18", "8-17": "8-17", "17-8": "8-17", "2-15": "2-15", "15-2": "2-15", "18-22": "18-22", "22-18": "18-22", "11-6": "11-6", "6-11": "11-6", "20-24": "20-24", "24-20": "20-24", "24-27": "24-27", "27-24": "24-27", "22-28": "22-28", "28-22": "22-28", "19-28": "19-28", "28-19": "19-28", "29-19": "29-19", "19-29": "29-19", "30-4": "30-4", "4-30": "30-4", "13-6": "13-6", "6-13": "13-6", "0-4": "0-4", "4-0": "0-4", "38-20": "38-20", "20-38": "38-20", "21-30": "21-30", "30-21": "21-30", "39-39": "39-39", "11-25": "11-25", "25-11": "11-25", "20-12": "20-12", "12-20": "20-12", "3-10": "3-10", "10-3": "3-10", "12-9": "12-9", "9-12": "12-9", "2-16": "2-16", "16-2": "2-16", "38-38": "38-38", "26-8": "26-8", "8-26": "26-8", "24-14": "24-14", "14-

In [ ]:
[{"key": "0", "gate_type": "Camping", "color": "#264653"}, {"key": "1", "gate_type": "Entrance", "color": "#2a9d8f"}, {"key": "2", "gate_type": "Gate", "color": "#8ab17d"}, {"key": "3", "gate_type": "General gate", "color": "#e9c46a"}, {"key": "4", "gate_type": "Ranger Base", "color": "#f4a261"}, {"key": "5", "gate_type": "Ranger Stop", "color": "#e76f51"}]

In [6]:
directions = [[1, 0], [0, -1], [-1, 0], [0, 1], [1, -1], [1, 1], [-1, -1], [-1, 1]]
edge_paths = {}
for path in tqdm(edge_set):
    start, end = path.split('-')
    start_r, start_c = gates_dict[gate_list[int(start)]]
    end_r, end_c = gates_dict[gate_list[int(end)]]
    
    seen = set()
    seen.add(gates_dict[gate_list[int(start)]])
    queue = deque([gates_dict[gate_list[int(start)]]])
    not_found = True
    
    best_path = {}
    best_path[gates_dict[gate_list[int(start)]]] = [(start_c, start_r)]
    
    while queue and not_found:
        r, c = queue.popleft()
        for dr, dc in directions:
            nr = r + dr
            nc = c + dc
            if (nr, nc) not in seen:
                seen.add((nr, nc))
                if nr == end_r and nc == end_c:
                    best_path[(nr, nc)]  = best_path[(r, c)].copy()
                    best_path[(nr, nc)].append((nc, nr))
                    not_found = False
                elif nr >= 0 and nr < 200 and nc >= 0 and nc < 200:
                    if p_norm[nr, nc] > 0.7:
                        queue.append((nr, nc))
                        if best_path.get((nr, nc)) is None:
                            best_path[(nr, nc)]  = best_path[(r, c)].copy()
                            best_path[(nr, nc)].append((nc, nr))
    edge_paths[path] = best_path[gates_dict[gate_list[int(end)]]]

100%|██████████| 122/122 [00:00<00:00, 162.39it/s]


In [7]:
size = 600
size_scale = size / 200
svg_path_dt = []
for edge in edge_paths.keys():
    svg_path = ['M ' + ' '.join([str(int(x * size_scale)) for x in edge_paths[edge][0]])]
    for l in edge_paths[edge][1:]:
        svg_path.append('L ' + ' '.join([str(int(x * size_scale)) for x in l]))
    svg_path_text = ' '.join(svg_path)
    svg_path_dt.append({"edge": edge, "path": svg_path_text})
svg_node_dt = []
for node in gates_dict2.keys():
    svg_node_dt.append({"node": node, 
                        "cx": str(int(gates_dict2[node][1] * size_scale)), 
                        "cy": str(int(gates_dict2[node][0] * size_scale)),
#                         "r": str(int(size_scale * 1.5)),
                        "t": str(gates_dict2[node][2])})

In [10]:
json.dumps(svg_path_dt)

'[{"edge": "20-14", "path": "M 345 450 L 345 447 L 345 444 L 342 441 L 342 438 L 342 435 L 342 432 L 345 429 L 345 426 L 345 423 L 345 420 L 342 417 L 342 414 L 342 411 L 342 408 L 342 405 L 345 402 L 345 399 L 342 396 L 339 393 L 339 390 L 336 387 L 336 384 L 333 381 L 330 378 L 327 375 L 324 372 L 324 369 L 321 366 L 321 363 L 318 360 L 318 357 L 315 354 L 312 351 L 309 348 L 309 345 L 306 342 L 303 339 L 300 336 L 297 333 L 297 330 L 294 327 L 291 324 L 291 321 L 288 318 L 288 315 L 285 312 L 285 309 L 282 306 L 282 303 L 282 300 L 279 297 L 279 294 L 279 291 L 279 288 L 279 285 L 279 282 L 279 279 L 279 276 L 276 273 L 273 270 L 270 270 L 267 267 L 267 264 L 270 261 L 270 258 L 270 255 L 273 252 L 273 249 L 273 246 L 273 243 L 273 240 L 270 237 L 267 234 L 264 231 L 264 228 L 261 225 L 258 222 L 258 219 L 255 216 L 255 213 L 258 210 L 258 207 L 258 204 L 258 201 L 258 198 L 255 195 L 255 192 L 252 189 L 249 186 L 246 183 L 243 180 L 243 177 L 243 174 L 240 171 L 237 168 L 234 165 L

In [11]:
json.dumps(svg_node_dt)

'[{"node": "camping0", "cx": "153", "cy": "120", "t": "0"}, {"node": "camping1", "cx": "384", "cy": "147", "t": "0"}, {"node": "camping2", "cx": "129", "cy": "189", "t": "0"}, {"node": "camping3", "cx": "132", "cy": "201", "t": "0"}, {"node": "camping4", "cx": "141", "cy": "264", "t": "0"}, {"node": "camping5", "cx": "57", "cy": "360", "t": "0"}, {"node": "camping6", "cx": "447", "cy": "528", "t": "0"}, {"node": "camping7", "cx": "540", "cy": "432", "t": "0"}, {"node": "camping8", "cx": "546", "cy": "141", "t": "0"}, {"node": "entrance0", "cx": "183", "cy": "36", "t": "1"}, {"node": "entrance1", "cx": "48", "cy": "198", "t": "1"}, {"node": "entrance2", "cx": "546", "cy": "258", "t": "1"}, {"node": "entrance3", "cx": "342", "cy": "498", "t": "1"}, {"node": "entrance4", "cx": "417", "cy": "549", "t": "1"}, {"node": "gate0", "cx": "186", "cy": "96", "t": "2"}, {"node": "gate1", "cx": "171", "cy": "129", "t": "2"}, {"node": "gate2", "cx": "69", "cy": "159", "t": "2"}, {"node": "gate3", "cx

In [26]:
with open('map.json', 'w') as f:
    f.writelines(json.dumps([svg_path_dict, svg_node_dict]))

In [27]:
bird_path = "M522 12013 c-35 -7 -62 -59 -62 -122 0 -48 -17 -158 -41 -269 -12 -54 -12 -221 0 -275 5 -23 15 -69 21 -102 29 -146 70 -235 199 -430 49 -74 103 -169 120 -209 29 -69 31 -82 29 -178 -2 -91 0 -107 19 -138 12 -19 29 -57 38 -85 19 -56 112 -237 176 -342 55 -89 134 -246 153 -304 9 -26 23 -115 31 -197 16 -157 38 -250 72 -299 11 -15 25 -44 32 -63 17 -52 77 -126 161 -198 179 -153 249 -224 381 -389 142 -178 173 -226 204 -323 27 -82 71 -165 108 -202 33 -34 124 -82 287 -153 123 -53 134 -58 255 -109 306 -131 355 -166 595 -431 70 -77 157 -165 194 -195 46 -38 81 -78 112 -130 25 -41 60 -89 76 -107 17 -17 28 -38 25 -45 -3 -7 -90 -114 -192 -238 -103 -124 -207 -252 -230 -285 -54 -75 -105 -172 -105 -197 0 -59 -52 -104 -257 -221 -73 -42 -163 -100 -200 -130 -84 -66 -148 -109 -328 -218 -217 -130 -412 -273 -696 -511 -42 -35 -108 -89 -149 -120 -115 -90 -201 -221 -204 -310 -1 -57 21 -71 104 -65 74 5 166 36 406 134 93 39 216 86 274 105 58 20 133 46 167 59 35 12 65 21 67 18 3 -2 -75 -85 -172 -184 -344 -350 -546 -605 -527 -664 14 -44 92 -34 184 23 32 19 69 39 82 45 28 10 212 171 404 351 179 169 292 258 511 404 104 70 224 157 267 195 57 52 96 77 150 98 39 15 98 45 129 65 137 89 250 130 264 95 8 -21 -26 -84 -105 -197 -37 -52 -84 -129 -105 -170 -22 -41 -85 -158 -141 -260 -133 -241 -199 -374 -220 -441 -58 -186 -78 -221 -280 -504 -146 -205 -243 -357 -365 -575 -50 -89 -219 -347 -367 -560 -163 -235 -325 -483 -376 -576 -61 -112 -78 -136 -323 -469 -254 -343 -332 -462 -433 -663 -72 -141 -88 -200 -69 -257 8 -25 20 -41 32 -43 42 -8 137 24 286 95 95 46 174 78 206 82 27 4 61 15 74 24 33 23 71 21 152 -8 59 -21 80 -24 137 -19 62 6 70 5 87 -15 69 -80 123 -106 275 -135 65 -12 79 -12 112 1 34 14 44 14 104 -1 l66 -17 52 27 c49 24 59 26 134 19 68 -6 86 -4 108 10 19 13 55 19 137 22 95 4 118 9 160 31 113 61 236 266 411 687 119 286 321 831 445 1205 42 126 169 489 188 535 19 49 78 181 142 320 18 39 54 157 81 264 52 207 90 299 145 354 19 18 82 60 141 91 125 68 215 130 368 257 61 50 169 136 240 192 72 56 198 155 280 221 144 115 279 208 338 234 50 22 24 33 332 -148 72 -42 243 -136 380 -208 l250 -131 140 -26 c324 -60 348 -66 449 -122 16 -9 79 -29 140 -45 61 -16 147 -40 191 -53 44 -13 124 -29 179 -35 171 -20 349 -77 451 -144 130 -86 267 -134 414 -146 134 -11 185 -29 256 -91 68 -59 78 -65 135 -74 57 -8 134 -32 183 -57 21 -10 104 -83 186 -161 81 -78 187 -170 235 -204 48 -34 134 -106 191 -159 312 -291 544 -368 679 -225 l29 30 -34 28 c-19 15 -65 57 -104 94 -38 36 -92 81 -119 99 -67 45 -188 162 -234 224 -76 105 -335 386 -403 439 -20 15 -42 27 -50 27 -42 0 -128 57 -222 147 -203 196 -413 301 -692 348 -84 14 -114 25 -185 65 -103 58 -245 119 -363 156 -49 15 -90 32 -92 38 -1 6 -8 29 -14 51 -37 136 -292 411 -447 482 -56 25 -94 52 -94 67 0 7 39 31 88 53 164 77 217 102 257 125 22 13 125 70 229 128 104 57 224 128 265 158 257 184 824 790 1036 1107 150 224 353 602 430 800 59 151 107 295 135 405 27 105 79 405 94 540 27 243 89 543 142 692 89 249 267 483 494 651 52 39 129 96 170 127 157 118 216 140 380 146 63 2 250 9 415 14 294 11 301 12 338 36 21 14 37 32 37 43 0 64 -305 198 -625 275 -133 32 -169 46 -255 96 -96 56 -161 119 -216 207 -133 217 -390 479 -649 663 -139 98 -377 188 -605 227 -123 21 -436 24 -535 4 -269 -52 -519 -165 -730 -329 -83 -64 -140 -120 -285 -277 -55 -60 -268 -250 -450 -402 -261 -218 -343 -284 -560 -453 -129 -101 -318 -244 -420 -318 -264 -194 -615 -509 -1135 -1022 -399 -393 -629 -638 -1106 -1179 -41 -47 -142 -177 -223 -289 -81 -111 -153 -202 -159 -202 -17 0 -172 141 -251 228 -85 93 -165 201 -245 330 -85 138 -92 182 -67 422 13 135 23 188 47 253 50 135 98 311 124 454 14 74 25 142 25 152 0 10 21 62 46 115 53 111 87 213 110 329 24 116 15 355 -16 452 -29 90 -81 194 -136 271 -53 75 -212 226 -233 222 -9 -2 -27 -25 -40 -51 l-23 -48 20 -57 c12 -31 34 -106 49 -167 26 -104 27 -123 28 -345 0 -207 -3 -249 -23 -350 -70 -345 -220 -683 -455 -1025 -115 -166 -126 -178 -198 -200 -46 -14 -93 -20 -165 -20 -91 0 -106 3 -157 29 -58 29 -120 96 -199 216 -40 60 -232 260 -315 328 -153 125 -247 324 -259 547 -8 165 -43 251 -147 365 -96 105 -173 244 -218 392 -26 86 -228 494 -420 851 -78 144 -147 272 -155 283 -32 51 -64 185 -78 326 -15 144 -16 150 -82 310 -37 90 -74 169 -82 176 -16 12 -20 13 -60 5z"
with open('bird.json', 'w') as f:
    f.writelines(json.dumps({'bird': bird_path}))

In [63]:
def write_svg(edge_paths, nodes, file, size=200):
    size_scale = size / 200
    svg_header = '<svg width="' + str(size) + '" height="' + str(size) + '" xmlns="http://www.w3.org/2000/svg">'
    svg_body = []
    for edge in edge_paths.keys():
        svg_path = ['M ' + ' '.join([str(int(x * size_scale)) for x in edge_paths[edge][0]])]
        for l in edge_paths[edge][1:]:
            svg_path.append('L ' + ' '.join([str(int(x * size_scale)) for x in l]))
        svg_path_text = ' '.join(svg_path)
        svg_body.append(
            '<path d="' + svg_path_text + '" id="' \
            + edge + '" stroke-width="' + str(int(2)) \
            + '" stroke="#CDCDCD" fill="transparent"/>'
        )
        
    node_color = {0:"#264653",1:"#2a9d8f",2:"#8ab17d",3:"#e9c46a",4:"#f4a261",5:"#e76f51"}
    for node in nodes.keys():
        svg_body.append(
            '<circle cx="' + str(int(nodes[node][1] * size_scale)) + 
            '" cy="' + str(int(nodes[node][0] * size_scale)) + 
            '" r="' + str(int(size_scale * 1.5)) + '" id="node-' 
            + node + '" fill="' + node_color[nodes[node][2]] + '"/>'
        )
    svg_content = svg_header + '\n' + '\n'.join(svg_body) + '\n' + '</svg>'
    with open(file, 'w') as f:
        f.writelines(svg_content)
    return

In [64]:
write_svg(edge_paths, gates_dict, 'demo/map.svg', 600)

IndexError: tuple index out of range

'<svg width="200" height="200200" xmlns="http://www.w3.org/2000/svg">\n<path d="M 64 25 L 63 26 L 63 27 L 63 28 L 63 29 L 64 30 L 64 31 L 64 32 L 65 33 L 66 34 L 66 35 L 67 36 L 67 37 L 68 38 L 68 39 L 69 40 L 69 41 L 70 42 L 70 43 L 71 44 L 71 45 L 72 46 L 73 47 L 74 48 L 74 49 L 75 50 L 76 51 L 76 52 L 77 53 L 77 54 L 78 55 L 78 56 L 79 57 L 80 58 L 81 59 L 81 60 L 81 61 L 82 62 L 83 63 L 84 64 L 85 65 L 85 66 L 86 67 L 86 68 L 86 69 L 86 70 L 86 71 L 85 72 L 85 73 L 86 74 L 86 75 L 87 76 L 88 77 L 88 78 L 89 79 L 90 80 L 91 81 L 91 82 L 91 83 L 91 84 L 91 85 L 90 86 L 90 87 L 90 88 L 89 89 L 90 90 L 91 90 L 92 91 L 92 92 L 93 93 L 93 94 L 93 95 L 93 96 L 93 97 L 93 98 L 93 99 L 93 100 L 94 101 L 94 102 L 94 103 L 95 104 L 95 105 L 96 106 L 96 107 L 97 108 L 97 109 L 98 110 L 99 111 L 99 112 L 100 113 L 101 114 L 102 115 L 103 116 L 103 117 L 104 118 L 105 119 L 106 120 L 106 121 L 107 122 L 107 123 L 108 124 L 108 125 L 109 126 L 110 127 L 111 128 L 112 129 L 112 130 L 113 131 L 114